In [1]:
import torch.nn.functional as F
import pytorch_lightning as pl
import pickle
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, random_split
from CustomDataModule import CustomDataModule
from LSTMModel import LSTMModel
from lightning.pytorch.loggers import WandbLogger
from pytorch_lightning.utilities.model_summary import summarize
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
def process_radar_return(radar_return):
    # Ensure the radar return has shape (num_rows, num_cols)
    assert len(radar_return.shape) == 2

    # Define the target length of 512
    target_length = 512

    # Randomly select the starting index for the sequence
    start_index = np.random.randint(0, radar_return.shape[0] - 4)

    # Select the subsequent 4 indices to form a sequence of 5 adjacent pulses
    selected_pulses = np.arange(start_index, start_index + 5)

    # Concatenate selected pulses along rows
    concatenated_pulses = np.concatenate([radar_return[pulse, :] for pulse in selected_pulses], axis=0)

    # Take np.abs to convert complex numbers to real numbers
    epsilon = 1e-10
    processed_radar_return = 10 * np.log10(((np.abs(concatenated_pulses))**2)+epsilon)

    # Ensure the processed radar return has length 512
    if processed_radar_return.shape[0] > target_length:
        # If length is greater than 512, truncate the vector
        processed_radar_return = processed_radar_return[:target_length]
    elif processed_radar_return.shape[0] < target_length:
        # If length is less than 512, pad with zeros
        min_value = np.min(processed_radar_return)
        pad_value = -200
        processed_radar_return = np.pad(processed_radar_return,
                                        (0, target_length - processed_radar_return.shape[0]),
                                        mode='constant', constant_values=pad_value)

    return processed_radar_return

In [3]:
def normalize_radar_return_column(combined_df):
    radar_data = combined_df['radar_return']

    # Flatten the radar data for scaler fitting
    flattened_data = [np.array(row).flatten() for row in radar_data]
    flattened_data = np.concatenate(flattened_data).reshape(-1, 1)

    # Fit the scaler on the flattened data
    scaler = MinMaxScaler()
    scaler.fit(flattened_data)

    # Transform each row separately
    normalized_data = [scaler.transform(np.array(row).reshape(-1, 1)).flatten().tolist() for row in radar_data]

    # Reshape the normalized data back to its original shape
    combined_df['radar_return'] = normalized_data

    return combined_df

In [4]:
data_dir = 'First_data'  # Change this to your data folder path
dataframes = []

# set device!
print(torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

True


In [5]:
print("creating the dataframe...")
for filename in os.listdir(data_dir):
    if filename.endswith('.pickle'):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, 'rb') as f:
            # Load data from pickle file
            data = pickle.load(f)
            # Extract radar_return and object_id
            radar_return = data['radar_return']
            object_id = data['object_id']
            # Concatenate radar_return along the columns
            concatenated_radar = process_radar_return(radar_return).astype('float32')
            # Create a DataFrame with concatenated radar and object_id
            df = pd.DataFrame({'radar_return': [concatenated_radar], 'object_id': [object_id]})
            # Append the DataFrame to the list
            dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df = normalize_radar_return_column(combined_df)

creating the dataframe...


In [6]:
class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(input_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.model(x)

In [7]:
class RadarDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


In [8]:
radar_data = torch.tensor(combined_df['radar_return'].values.tolist(), dtype=torch.float32)
object_ids = combined_df['object_id'].values.tolist()
# Use LabelEncoder to convert object_ids to numerical labels
label_encoder = LabelEncoder()
object_ids_encoded = label_encoder.fit_transform(object_ids)

# Convert encoded labels to tensor
object_ids_tensor = torch.tensor(object_ids_encoded, dtype=torch.float32)

# Define hyperparameters
input_size = radar_data.shape[1]
hidden_size = 512
output_size = 1
learning_rate = 0.001
num_epochs = 20
batch_size = 32

# Create FNN instance
model = FNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoader for batch training
dataset = RadarDataset(radar_data, object_ids_tensor)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU if available
model.to(device)

FNN(
  (model): Sequential(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=512, out_features=512, bias=True)
    (10): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Linear(in_features=512, out_features=512, bias=True)
    (16): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True,

In [10]:

# Training loop
for epoch in range(num_epochs):
    correct_predictions = 0
    total_samples = 0
    total_loss = 0.0
    
    for i, (inputs, labels) in enumerate(data_loader):
        # Move inputs and labels to the same device as the model
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels.view(-1, 1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

        # Accumulate total loss
        total_loss += loss.item()

        if (i+1) % 100 == 0:
            avg_loss = total_loss / 100
            accuracy = correct_predictions / total_samples

            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(data_loader)}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')

            total_loss = 0.0
            correct_predictions = 0
            total_samples = 0

print('Training finished.')

Epoch [1/20], Step [100/625], Loss: 10.7131, Accuracy: 0.0934
Epoch [1/20], Step [200/625], Loss: 8.7211, Accuracy: 0.1091
Epoch [1/20], Step [300/625], Loss: 8.6818, Accuracy: 0.1053
Epoch [1/20], Step [400/625], Loss: 8.4037, Accuracy: 0.0963
Epoch [1/20], Step [500/625], Loss: 8.5972, Accuracy: 0.1025
Epoch [1/20], Step [600/625], Loss: 8.4514, Accuracy: 0.0944
Epoch [2/20], Step [100/625], Loss: 8.5812, Accuracy: 0.0978
Epoch [2/20], Step [200/625], Loss: 8.5369, Accuracy: 0.1019
Epoch [2/20], Step [300/625], Loss: 8.4149, Accuracy: 0.1016
Epoch [2/20], Step [400/625], Loss: 8.5161, Accuracy: 0.0972
Epoch [2/20], Step [500/625], Loss: 8.5160, Accuracy: 0.1009
Epoch [2/20], Step [600/625], Loss: 8.2908, Accuracy: 0.1019
Epoch [3/20], Step [100/625], Loss: 8.6891, Accuracy: 0.0984
Epoch [3/20], Step [200/625], Loss: 8.3897, Accuracy: 0.0941
Epoch [3/20], Step [300/625], Loss: 8.5592, Accuracy: 0.1081
Epoch [3/20], Step [400/625], Loss: 8.4786, Accuracy: 0.1062
Epoch [3/20], Step [500